In [69]:
%pylab inline

from __future__ import division

import copy
import os 
import argparse
import numpy as np 

from astropy.io import fits 
from astropy    import wcs 

import lsst.daf.persistence   as dafPersist
import lsst.afw.coord         as afwCoord
import lsst.afw.image         as afwImage
import lsst.afw.geom          as afwGeom
import lsst.afw.table         as afwTable

# Matplotlib default settings
import matplotlib as mpl 
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = 12, 10
mpl.rcParams['xtick.major.size'] = 8.0
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['xtick.minor.size'] = 4.0
mpl.rcParams['xtick.minor.width'] = 1.5
mpl.rcParams['ytick.major.size'] = 8.0
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['ytick.minor.size'] = 4.0
mpl.rcParams['ytick.minor.width'] = 1.5
mpl.rc('axes', linewidth=2)

# Shapely related imports 
from shapely.geometry import MultiPolygon, Point
from shapely.geometry import Polygon, LineString
from shapely          import wkb 
from shapely.ops      import cascaded_union 

from scipy import ndimage
from skimage.measure import find_contours, approximate_polygon    

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [87]:
def showHscMask(coords, large=None, title='No Data Mask Plane', 
                pngName=None):
    
    from matplotlib.patches import Polygon as mpPoly
    
    fig = plt.figure(figsize=(10, 10), dpi=120)

    ax = fig.add_subplot(111)
    fontsize = 14
    ax.minorticks_on()

    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        
    # Set title
    ax.set_title(title, fontsize=25, fontweight='bold')
    ax.title.set_position((0.5,1.01))

    # Outline all the mask regions
    for raDec in coords:
        ax.plot(raDec[:, 1], raDec[:, 0], '-r', linewidth=1.5)
    
    # Using polygon to highlight all the large ones
    if large is not None:
        for raDec in large:
            ax.plot(raDec[:, 1], raDec[:, 0], '-b', linewidth=2.0)
                
    fig.subplots_adjust(hspace=0.1, wspace=0.1,
                        top=0.95, right=0.95)
    
    if pngName is not None:     
        fig.savefig(pngName)
        plt.close(fig)

In [88]:
def imgAddNoise(im, gaussian, factor): 
    
    im = ndimage.gaussian_filter(im, gaussian)
    im += factor * np.random.random(im.shape)
    
    return im 

In [89]:
def getPixelRaDec(wcs, xx, yy): 
    
    coord = wcs.pixelToSky(xx, yy).toIcrs()
        
    ra  = coord.getRa().asDegrees()
    dec = coord.getDec().asDegrees()
    
    return ra, dec

In [90]:
def polySaveWkb(poly, wkbName):

    polyWkb = wkb.dumps(poly)

    wkbFile = open(wkbName, 'w')
    wkbFile.write(polyWkb.encode('hex'))
    wkbFile.close()

In [17]:
# Save the Polygon region into a DS9 .reg file
def getPolyLine(polyCoords):

    coordShow = map(lambda x: str(x[0]) + ' ' + str(x[1]) + ' ', polyCoords)

    # The format for Polygon in DS9 is:
    # Usage: polygon x1 y1 x2 y2 x3 y3 ...
    polyLine = 'polygon '
    for p in coordShow:
        polyLine += p

    polyLine += '\n'

    return polyLine

# !!! Make sure that the Polygon is simple
def polySaveReg(poly, regName, listPoly=False, color='blue'):

    # DS9 region file header
    head1 = '# Region file format: DS9 version 4.1\n'
    head2 = 'global color=%s width=2\n' % color
    head3 = 'icrs\n'
    # Open the output file and write the header
    regFile = open(regName, 'w')
    regFile.write(head1)
    regFile.write(head2)
    regFile.write(head3)

    if listPoly:
        #for pp in poly:
        for i in range(len(poly)):
            pp = poly[i]
            print i
            if pp.geom_type is "Polygon":
            # Get the coordinates for every point in the polygon
                polyCoords = pp.boundary.coords[:]
                polyLine = getPolyLine(polyCoords)
                regFile.write(polyLine)
            elif pp.geom_type is "MultiPolygon":
                for mm in pp.geoms: 
                    polyCoords = mm.boundary.coords[:]
                    polyLine = getPolyLine(polyCoords)
                    regFile.write(polyLine)
    else:
        if poly.geom_type is "Polygon":
            polyCoords = poly.boundary.coords[:]
            polyLine = getPolyLine(polyCoords)
            regFile.write(polyLine)
        elif poly.geom_type is "MultiPolygon":
            for mm in pply.geoms: 
                polyCoords = mm.boundary.coords[:]
                polyLine = getPolyLine(polyCoords)
                regFile.write(polyLine)            

    regFile.close()

In [92]:
def coaddPatchNoData(rootDir, tract, patch, filter, prefix='hsc_coadd', 
                     savePNG=True, verbose=True, tolerence=4, 
                     minArea=10000): 
    
    # Make a butler and specify the dataID 
    butler = dafPersist.Butler(rootDir)
    dataId = {'tract':tract, 'patch':patch, 'filter':filter}
    
    # Get the name of the input fits image
    if rootDir[-1] is '/': 
        fitsName = rootDir + 'deepCoadd/' + filter + '/' + str(tract).strip() + '/' + patch + '.fits'
    else: 
        fitsName = rootDir + '/deepCoadd/' + filter + '/' + str(tract).strip() + '/' + patch + '.fits'
    if not os.path.isfile(fitsName): 
        raise Exception('Can not find the input fits image: %s' % fitsName)
    # Read in the original image and get the wcs 
    # TODO: This is not perfect
    hduList = fits.open(fitsName)
    header = hduList[1].header 
    imgWcs = wcs.WCS(header)
        
    # Get the name of the wkb and deg file
    strTractPatch = (str(tract).strip() + '_' + patch + '_' + filter)
    ## For all the accepted regions 
    noDataAllWkb = prefix + '_' + strTractPatch + '_nodata_all.wkb'
    noDataAllReg = prefix + '_' + strTractPatch + '_nodata_all.reg'
    ## For all the big mask regions 
    noDataBigWkb = prefix + '_' + strTractPatch + '_nodata_big.wkb'
    noDataBigReg = prefix + '_' + strTractPatch + '_nodata_big.reg'    
    
    # Get the name of the png file
    titlePng = prefix + strTractPatch + '_NODATA'
    noDataPng = prefix + '_' + strTractPatch + '_nodata.png'
    
    if verbose: 
        print "## Reading Fits Image: %s" % fitsName
        
    # Get the exposure from the butler 
    calExp = butler.get('deepCoadd', dataId, immediate=True)
    
    # Get the object for mask plane 
    mskImg = calExp.getMaskedImage().getMask() 
    
    # Extract the NO_DATA plane 
    # TODO: NO_DATA is not a system mask, maybe should use INTRP later
    noData = copy.deepcopy(mskImg)
    noData &= noData.getPlaneBitMask('NO_DATA')
    # Return the mask image array 
    noDataArr = noData.getArray()
    
    # Set all masked pixels to be 1
    noDataArr /= 256
    # Pad the 2-D array by a little 
    noDataArr = np.lib.pad(noDataArr, ((1, 1), (1, 1)), 'constant', constant_values=0)    
    
    # Try a very different approach: Using the find_contours and 
    # approximate_polygon methods from scikit-images package
    maskShapes = []  # For all the accepted mask regions 
    maskCoords = []  # For the "corner" coordinates of these regions
    maskAreas  = []  # The sizes of all regions
    
    # Only find the 0-level contour 
    contoursAll = find_contours(noDataArr, 0)
    if verbose: 
        print "### %d contours have been detected" % len(contoursAll)
    for maskContour in contoursAll:
        # Approximate one extracted contour into a polygon
        # tolerance decides the accuracy of the polygon, hence 
        # the number of coords for each polygon.  
        # Using large tolerance also means smaller number of final 
        # polygons
        contourCoords = approximate_polygon(maskContour, tolerance=tolerence)
        # Convert these coordinates into (RA, DEC) using the WCS information 
        contourSkyCoords = map(lambda x: [x[1], x[0]], contourCoords)
        contourRaDec     = imgWcs.wcs_pix2world(contourSkyCoords, 1)
        # Require that any useful region must be at least an triangular 
        if len(contourCoords) >= 3: 
            # Form a lineString using these coordinates 
            maskLine = LineString(contourRaDec)
            # Check if the lineString is valid and simple, so can be used 
            # to form a closed and simple polygon
            if maskLine.is_valid and maskLine.is_simple: 
                contourPoly = Polygon(contourRaDec)
                maskShapes.append(contourPoly)
                maskCoords.append(contourRaDec)
                maskAreas.append(Polygon(contourCoords).area)
                
    if verbose: 
        print "### %d regions are useful" % len(maskAreas)
    # Save all the masked regions to a .reg file 
    polySaveReg(maskShapes, noDataAllReg, listPoly=True)
    # Also create a MultiPolygon object, and save a .wkb file 
    maskAll = cascaded_union(maskShapes)
    polySaveWkb(maskAll, noDataAllWkb)
    
    # Isolate the large ones
    maskBigList = np.array(maskShapes)[np.where(np.array(maskAreas) > minArea)]
    maskBigList = map(lambda x: x, maskBigList)
    coordBigList = np.array(maskCoords)[np.where(np.array(maskAreas) > minArea)]
    nBig = len(maskBigList)
    if nBig > 0: 
        if verbose: 
            print "### %d regions are larger than the minimum mask sizes" % nBig
        # Save all the masked regions to a .reg file 
        polySaveReg(maskBigList, noDataBigReg, listPoly=True)
        # Also create a MultiPolygon object, and save a .wkb file 
        maskBig = cascaded_union(maskBigList)
        polySaveWkb(maskBig, noDataBigWkb)  
    else: 
        maskBig = None
        if verbose: 
            print "### No region is larger than the minimum mask sizes"
        
    if savePNG:
        showHscMask(maskCoords, large=coordBigList, title=titlePng, pngName=noDataPng)
                
    return maskAll, maskBig

In [98]:
# Example 

rootDir = '/lustre/Subaru/SSP/rerun/yasuda/SSP3.4.1_20141224/'
tract   = 8280
patch   = '2,2' 
filter  = 'HSC-I'

In [99]:
maskAll, maskBig = coaddPatchNoData(rootDir, tract, patch, filter, prefix='hsc_coadd', 
                                    savePNG=True, verbose=True, tolerence=4, minArea=10000)

## Reading Fits Image: /lustre/Subaru/SSP/rerun/yasuda/SSP3.4.1_20141224/deepCoadd/HSC-I/8280/2,2.fits
### 314 contours have been detected
### 109 regions are useful
### 2 regions are larger than the minimum mask sizes


In [106]:
def listAllImages(rootDir, filter):

    import glob

    if rootDir[-1] is '/':
        searchDir = rootDir + 'deepCoadd/' + filter.upper() + '/*/*.fits'
    else:
        searchDir = rootDir + '/deepCoadd/' + filter.upper() + '/*/*.fits'

    return map(lambda x: x, glob.glob(searchDir))

In [35]:
def batchPatchNoData(rootDir, filter='HSC-I', prefix='hsc_coadd'): 
    
    # Get the list of coadded images in the direction 
    imgList = listAllImages(rootDir, filter) 
    imgList = imgList[0:3] ### Test
    
    # Get the list of tract and patch for these images 
    tract = map(lambda x: int(x.split('/')[-2]), imgList)
    patch = map(lambda x: x.split('/')[-1].split('.')[0], imgList)
    
    results = map(lambda x, y: coaddPatchNoData(rootDir, x, y, filter, 
                                prefix=prefix, savePNG=True, verbose=True, tolerence=4,
                                minArea=10000), tract, patch)
    allList = map(lambda x: x[0], results)
    bigList = map(lambda x: x[1], results)
    
    allUse = [] 
    for ss in allList: 
        if ss is not None: 
            allUse.append(ss)
    bigUse = [] 
    for tt in bigList: 
        if tt is not None: 
            bigUse.append(tt)
            
    # Make a cascaded union of them 
    allComb = cascaded_union(allUse)
    bigComb = cascaded_union(bigUse)
    
    # Save these polygons as a .wkb file 
    polySaveWkb(allComb, prefix + '_' + filter + '_nodata_all_combined.wkb')
    polySaveWkb(bigComb, prefix + '_' + filter + '_nodata_big_combined.wkb')
    
    # Break them down into list 
    ## ALL 
    if len(allUse) > 0: 
        polySaveReg(allUse, prefix + '_' + filter + '_nodata_all_combined.reg', 
                    listPoly=True, color='red')
    ## BIG
    if len(bigUse) > 0:
        polySaveReg(bigUse, prefix + '_' + filter + '_nodata_big_combined.reg', 
                    listPoly=True, color='blue')
    
    return allComb, bigComb

In [2]:
# Read a .wkb file into a Polygon shape
def polyReadWkb(wkbName, load=True):

    wkbFile = open(wkbName, 'r')
    polyWkb = wkbFile.read().decode('hex')
    wkbFile.close()

    if load is True:
        return wkb.loads(polyWkb)
    else:
        return polyWkb

In [24]:
test1 = polyReadWkb('/home/song/work/early/ssp341_wide_nodata/ssp341_wide_9451_2,8_HSC-G_nodata_all.wkb').geoms[:]
test2 = polyReadWkb('/home/song/work/early/ssp341_wide_nodata/ssp341_wide_9451_2,6_HSC-G_nodata_all.wkb').geoms[:]
test3 = polyReadWkb('/home/song/work/early/ssp341_wide_nodata/ssp341_wide_9451_2,7_HSC-G_nodata_all.wkb').geoms[:]

In [81]:
#files = ['/home/song/work/early/ssp341_wide_nodata/ssp341_wide_9451_2,8_HSC-G_nodata_all.wkb', 
#         '/home/song/work/early/ssp341_wide_nodata/ssp341_wide_9451_2,7_HSC-G_nodata_all.wkb', 
#         '/home/song/work/early/ssp341_wide_nodata/ssp341_wide_9451_2,6_HSC-G_nodata_all.wkb']
files = ['/home/song/work/early/ssp341_cosmos_nodata/ssp341_cosmos_0_HSC-G_nodata_all_wkb.wkb']

In [82]:
temp = []
for f in files: 
    geoms = polyReadWkb(f).geoms[:]
    for g in geoms: 
        temp.append(g)

In [83]:
m = cascaded_union(temp)

In [112]:
m[1].boundary.coords[:]

[(150.93634600593458, 2.203149695207256),
 (150.94545041307933, 2.202631198051483),
 (150.97103650018613, 2.2022429412464475),
 (150.9451698378152, 2.2018847768753727),
 (150.9367652097121, 2.201376317796325),
 (150.93335698855435, 2.2017515501287455),
 (150.92924439045134, 2.2018662548760792),
 (150.9156612029758, 2.202041468769308),
 (150.9138403690229, 2.2022757896242977),
 (150.92152445243738, 2.202524387128262),
 (150.92154461993135, 2.202598124755066),
 (150.92699955925343, 2.202701519235631),
 (150.92943552884753, 2.2027803283698066),
 (150.9307428779161, 2.2027795881924677),
 (150.93074322960956, 2.202772477834623),
 (150.93349767287359, 2.2028246863402936),
 (150.93634600593458, 2.203149695207256)]

In [85]:
mask = polyReadWkb(files[0])

In [86]:
a = mask.geoms[2672].boundary

In [113]:
ra, dec = 150.939, 2.2028

In [114]:
mask.contains(Point(ra, dec))

True